Yes, sounds like a plan. Zach, I can point you to the late-2018 Spire data as well.
 

On the parallel/recent-data effort, let’s choose one single high-inclination Spire satellite that is at similar altitude to IceSat2 during the late-2019-to-early-2021 time period. A similar LT would also be good. I don’t recall how long it takes IceSat to precess 12 hours in local time, but most of the high-inclination Spire satellites seem to be sun-sync, so we may want to identify a shorter time period of interest, and then select which Spire satellite is at a nearby LT to IceSat2 at that particular time. Zach, you have the spreadsheet from Vu that has the LT of the Spire orbits, so we can start there. Do we have a time history of IceSat2’s LT of ascending or descending nodes?



In [1]:
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
import gc

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN_Plot import *


# Attempt from Den File (LLA)

In [3]:
global_path = '/data/zach_work/Paper2022_assessment/runsetting_PhysicalCDasFixedValue/'
dir_save = '/data/data_geodyn/results/icesat2/Paper2022_PhysicalCD/SavedResults/'
dir_save_camel = '/data/data_geodyn/results/icesat2/Paper2022_PhysicalCD/CAMEL/icesat2_modelassessment_fixedcd/'
plot_dir = '/data/zach_work/Paper2022_assessment/plots/'

#####----------------------------------------------------------------------------------
run_list = [  
#                'msis2'      ,   
#                'tiegcm_oc'  ,
               'jb2008'     ,
#                'hasdm_oc'   ,
#                'gitm'       ,           
#                'dtm2020_o'  ,
#                'ctipe_oc'   ,
           ]



#####----------------------------------------------------------------------------------
##### Choose a Stage:
# STAGE = '1_Calculate_DRIA_CD'
STAGE = '2_RunGEODYN_with_PhysicalFixedCD'
# STAGE = '3_RunGEODYN_AdjustCDforBias'
# STAGE = '4_GetMeanCDfromAdjustment'
# STAGE = '5_RunGEODYN_FixedMeanCD'



#####----------------------------------------------------------------------------------
##### Choose if GEODYN should be Re-run:
# Logic_RUN = True
Logic_RUN = False
#####----------------------------------------------------------------------------------
save_plot_flag = False
#####----------------------------------------------------------------------------------
Logic_makeCSV = False

####
Logic_read_pickles = False


In [4]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
import gc
import os

if Logic_read_pickles:
    sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
    from PYGEODYN import Pygeodyn


    if STAGE == "1_Calculate_DRIA_CD":
        settings_dir = 'DRIAruns/'
        runtype = '_DRIA.yaml'
        pickleName= '_2week_DRIA.pkl'


    elif STAGE== "2_RunGEODYN_with_PhysicalFixedCD":
        runtype = '_BWDRAG_Cd2.5.yaml'    
        settings_dir = 'CD_2.5/'
        pickleName= '_2week_CD2.5.pkl'


    elif STAGE== "3_RunGEODYN_AdjustCDforBias":
        runtype = '_BWDRAG_adjustCD.yaml'    
        settings_dir = 'Adjust_for_bias/'
        pickleName= '_2week_CDadjust.pkl'

    elif STAGE== "4_GetMeanCDfromAdjustment":
        runtype = '_BWDRAG_adjustCD.yaml'    
        settings_dir = 'Adjust_for_bias/'
        pickleName= '_2week_CDadjust.pkl'

    elif STAGE== "5_RunGEODYN_FixedMeanCD":
        runtype = '_BWDRAG_FixedMeanCD.yaml'    
        settings_dir = 'Adjust_for_bias/'
        pickleName= '_2week_FixedMeanCD.pkl'


    else:
        sys.exit(0)

    #####################################################
    ###### RUN GEODYN
    ###### -----------------------------------------------
    if Logic_RUN:
        for i,model in enumerate(run_list):   
            run_settings = global_path+settings_dir+ ''+model+runtype
            ### Load the data into an object
            RunObject = Pygeodyn(run_settings)
            RunObject.RUN_GEODYN()
            RunObject = 0
            gc.collect() 
    # sys.exit(0)

    ###### -----------------------------------------------


    for i,model in enumerate(run_list):     

        pickle_file = dir_save+model+pickleName

        if not os.path.exists(pickle_file):
            print('Must create pickle file...')
            print('   ',  pickle_file)
            print('   ', 'Reading Geodyn Data')

            run_settings = global_path+settings_dir+ ''+model+runtype
            ### Load the data into an object
            Obj = Pygeodyn(run_settings)
            Obj.getData_BigData_lowmemory()
            gc.collect()

            #### Pickle the object to save it
            print('   ', 'Saving pickle')
            filehandler = open(pickle_file, 'wb') 
            pickle.dump(Obj, filehandler)
            filehandler.close()
            Obj = 0
            print('   ', 'Saved pickle')

    Obj_Geodyn = {}
    for i,model in enumerate(run_list):     
        ### Load the data if the pickles exist
        print()
        print()
        gc.collect()

        pickle_file = dir_save+model + pickleName

        filehandler = open(pickle_file, 'rb') 
        Obj_Geodyn[model] = pickle.load(filehandler)
        filehandler.close()
        print('Loaded data from pickle... ', model)


    gc.collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Obj_Geodyn['jb2008'].__dict__.keys()

In [6]:
# Obj_Geodyn['jb2008'].__dict__['Density']['2018.313.01']

In [7]:
csv_dicts = {}

for imodel in run_list:
#     obj_m1 = Obj_Geodyn[imodel]
    filename = dir_save_camel+'TwoWeeks_FixedCD2.5_'+imodel+'.csv'


    csv_dicts[imodel] = pd.read_csv(filename, sep=',')



In [14]:
pd.to_datetime(csv_dicts[imodel]['Date'])[0].hour

0

In [65]:
calc_slt = []

for it, timeval in enumerate(csv_dicts[imodel]['Date']):      
    time = pd.to_datetime(timeval)
    
    lon = csv_dicts[imodel]['Lon'][it] #lon_array
    UT = time.hour  
#     print(UT)
    calc_slt.append(np.mod(lon/15 + UT, 23))
    

        
        
        
csv_dicts[imodel]['LocTime'] = calc_slt

In [66]:
# csv['Date']

In [67]:
# UT = 12

# # Figure out the timing stuff within TIEGCM 
# ut_index = np.where(tiegcm['ut']==UT)[0][0]  # for tiegcm indexing
# calc_slt = np.mod(tiegcm['lon']/15 + tiegcm['ut'][ut_index], 24)
# slt  = np.sort(calc_slt)
# islt = np.argsort(calc_slt)



In [68]:
mask_day1 = pd.to_datetime(csv_dicts[imodel]['Date'].values).day==9

mask_orbits = pd.to_datetime(csv_dicts[imodel]['Date'].values).hour<3


csv_dicts[imodel]['Date'][mask_day1 & mask_orbits]

0     2018-11-09 00:03:00
1     2018-11-09 00:05:00
2     2018-11-09 00:07:00
3     2018-11-09 00:09:00
4     2018-11-09 00:11:00
             ...         
84    2018-11-09 02:51:00
85    2018-11-09 02:53:00
86    2018-11-09 02:55:00
87    2018-11-09 02:57:00
88    2018-11-09 02:59:00
Name: Date, Length: 89, dtype: object

In [69]:
%load_ext autoreload
%autoreload 2
        
        
def plot_from_csv(fig, plot_num, csv, model_name):
    ####  Get Plot Parameters for this model
    model_m1 = model_name
    col = get_plot_params(plot_num, model_m1)

    
    mask_day1 = pd.to_datetime(csv_dicts[imodel]['Date'].values).day==9
    mask_day_last = pd.to_datetime(csv_dicts[imodel]['Date'].values).day==22

    
    mask_orbits = pd.to_datetime(csv_dicts[imodel]['Date'].values).hour<3
   
    
    fig.add_trace(go.Scattergl(x=csv['Lon'][mask_day1 & mask_orbits],
                             y=csv['Lat'][mask_day1 & mask_orbits],
                                 mode='markers+text',
                                 opacity=1,
                                     marker=dict(color='blue',size=3),
                                     line = dict(color='blue',width=2),
                                 showlegend=False),
                                 secondary_y=False, row=1, col=1)
    fig.add_trace(go.Scattergl(x=csv['LocTime'][mask_day1 & mask_orbits],
                             y=csv['Lat'][mask_day1 & mask_orbits],
                                 mode='markers',
                                 opacity=1,
                                 marker=dict(color='blue',size=3),
                                 showlegend=False),row=2, col=1)
   

    ### -----------------------------------------------------------------------------------------------------        

    fig.add_trace(go.Scattergl(x=csv['Lon'][mask_day_last & mask_orbits],
                             y=csv['Lat'][mask_day_last & mask_orbits],
                                 mode='markers+text',
                                 opacity=1,
                                     marker=dict(color='orange',size=3),
                                     line = dict(color='orange',width=2),
                                 showlegend=False),
                                 secondary_y=False, row=1, col=1)
    fig.add_trace(go.Scattergl(x=csv['LocTime'][mask_day_last & mask_orbits],
                             y=csv['Lat'][mask_day_last & mask_orbits],
                                 mode='markers',
                                 opacity=1,
                                 marker=dict(color='orange',size=3),
                                 showlegend=False),row=2, col=1)
#     if plot_num == 0:
    fig.update_yaxes( title="Latitude", 
                     exponentformat= 'power',row=1, col=1)
    fig.update_xaxes( title="Longitude", 
                         exponentformat= 'power',row=1, col=1)
    fig.update_yaxes( title="Latitude", 
                     exponentformat= 'power',row=2, col=1)
    fig.update_xaxes( title="Local Solar Time", 
                         exponentformat= 'power',row=2, col=1)

    return(fig)
    ### -----------------------------------------------------------------------------------------------------
    ### -----------------------------------------------------------------------------------------------------
    ### -----------------------------------------------------------------------------------------------------
    ### -----------------------------------------------------------------------------------------------------
    ### -----------------------------------------------------------------------------------------------------

run_dict = { 
              'jb2008'   : 3  ,
           }

fig = make_subplots(rows=2, cols=1,
                    vertical_spacing = 0.12,
                    shared_xaxes=False)


rms_model = {}

for iplot,model in enumerate(run_list):
    fig = plot_from_csv(fig, iplot, csv_dicts[model], model )




#######################################################
font_dict=dict(family='Arial',size=16,color='black')
######################################################
fig.update_layout(
                  autosize=False,    width=1000,    height=800,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                 )
fig.update_annotations(font_size=16)  # Increase size of subplot title
fig.show(config=dict({
            'displayModeBar': False,
            'responsive': False,
            'staticPlot': False,
            'displaylogo': False,
            'showTips'   : False,
            }))



# longitude, LT, Change UT to be based on the date
        


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
rate = (.7-1.4)/14 
print('LT', )

In [12]:
import numpy as np

## ICESat-2

In [13]:
R_E = 6378137 #meters
a   = 6855953.9
e  = 0.001398
mu = 3.986004418*1e14 #m^3 s−2
Period = (2*np.pi)*np.sqrt((a**3)/mu)
ang_vel  = (2*np.pi)/Period   #2π radians divided by its period in seconds
i  = 92.0013*(np.pi/180) #deg

J2  = 1.08262668*1e-3


precession_rate = (-3/2)* ( (R_E**2) / (a*(1-e**2))**2 ) * J2 * ang_vel * np.cos(i)
print(precession_rate, 'rads/sec')
precession_rate_degs_day = precession_rate*(180/np.pi)  * (86400)
print(precession_rate_degs_day, 'degs/days')


print()

print(360/(precession_rate_degs_day)," days to complete one full rotation")

5.458733541399004e-08 rads/sec
0.270226707905086 degs/days

1332.214727370493  days to complete one full rotation


# Attempt from PSO

In [14]:
# mu = 3.986004415*10**5
# radius = 6378.1363      # km



# def RV_to_OE(R_vec, V_vec, mu):
#     import numpy as np
#     R_vec = np.array(R_vec) #km
#     V_vec = np.array(V_vec )
# #     r = np.sqrt(R_vec[0]**2 + R_vec[1]**2 + R_vec[2]**2) 
# #     v = np.sqrt(V_vec[0]**2 + V_vec[1]**2 + V_vec[2]**2) 
#     r = np.linalg.norm(R_vec)
#     v = np.linalg.norm(V_vec)



#     h_vec = np.array(np.cross(R_vec,V_vec))
#     h = np.linalg.norm(h_vec)
    
#     p = (h**2/mu)

#     n_vec = np.cross(np.array([0,0,1]),h_vec)
#     n = np.linalg.norm(n_vec)


#     energy = ((v**2)/2)-(mu/r)

#     e_vec = (((v**2)-(mu/r))*R_vec - (np.dot(R_vec,V_vec))*V_vec)/mu

#     a = -1*(mu/(2*energy))
#     e = np.linalg.norm(e_vec)
#     i = np.arccos(h_vec[2] / h)*(180/np.pi)

#     RAAN = np.arccos(n_vec[0]/n)*(180/np.pi)
#     if n_vec[1]<0:
#         RAAN = -1*RAAN

#     arg_peri = np.arccos(np.dot( n_vec, e_vec )/ (n * e) )*(180/np.pi)
#     if e_vec[2]<0:
#         arg_peri = -1*arg_peri

#     true_anom = np.arccos(np.dot(e_vec,R_vec)/(e*r))*(180/np.pi)
#     if np.dot(R_vec,V_vec) < 0:
#         true_anom = -1*true_anom
    
#     vr= (mu/h)*(e*np.sin(true_anom*(np.pi/180)))

#     v_theta = (mu/h)*(1 + e*np.cos(true_anom*(np.pi/180)))

#     r_vec_rot = np.array([r, 0,0])
#     v_vec_rot = np.array([vr, v_theta,0])
    
#     Period = (2*np.pi)*np.sqrt(a**3/mu)
    
#     dict_OE = {}
#     dict_OE['r_mag']          = r
#     dict_OE['v_mag']          = v
#     dict_OE['a']              = a
#     dict_OE['ecc']            = e
#     dict_OE['energy_spec']    = energy
#     dict_OE['h']              = h
#     dict_OE['p']              = p
#     dict_OE['Period']         = Period
#     dict_OE['inc']            = i
#     dict_OE['arg_periapsis']  = arg_peri
#     dict_OE['raan']           = RAAN
#     dict_OE['true_anom']      = true_anom
#     dict_OE['r_vec_RotFrame'] = r_vec_rot
#     dict_OE['v_vec_RotFrame'] = v_vec_rot
        
# #     print('r_mag =' ,r)
# #     print('v_mag =' ,v)
# #     print('a ='     ,a)
# #     print('ecc ='   ,e)
# #     print('energy_spec =',energy)
# #     print('h =',h)
# #     print('p =', p)
# #     print('Period =', Period)
# #     print('inc =',i)
# #     print('arg_periapsis =',arg_peri)
# #     print('raan =', RAAN)
# #     print('true_anom =', true_anom)
# #     print('r_vec_RotFrame =', r_vec_rot)
# #     print('v_vec_RotFrame =',v_vec_rot)
#     return(dict_OE)


In [15]:
# # Load PCE Data:
# OrbitData = "/data/data_geodyn/inputs/icesat2/setups/PCE_ascii.txt"
    


    
# epoch_start = '181109000000.0000000'
# epoch_end   = '181110000000.0000000'

In [16]:
# epoch_start_YYMMDD     = epoch_start[:6].strip() 
# epoch_start_HHMM       = epoch_start[7:11].strip()
# epoch_start_dt = pd.to_datetime( epoch_start_YYMMDD+epoch_start_HHMM, format='%y%m%d%H%M%S')
# epoch_end_YYMMDD     = epoch_end[:6].strip() 
# epoch_end_HHMM       = epoch_end[7:11].strip()

# epoch_start_dt_STR = str(epoch_start_dt)
# epoch_end_dt = pd.to_datetime( epoch_end_YYMMDD+epoch_end_HHMM, format='%y%m%d%H%M%S')

# date_in_file_flag = False

# print("Epoch Start: "  , epoch_start_dt_STR)
# print("Epoch End:   "  , epoch_end_dt)

# with open(OrbitData, 'r') as f:
#     for line_no, line_text in enumerate(f):

#         if epoch_start_dt_STR in line_text:
#             date_in_file_flag= True
#             print('    ','xyzline',line_no,line_text)

#             break


# DF_orbit = pd.read_csv(OrbitData, 
# #             skiprows = line_no, 
# #             nrows=1,           
#             sep = '\s+',
#             dtype=str,
#             names = [
#                 'Date',
#                 'MJDSECs', 
#                 'RSECS', #(fractional secs)
#                 'GPS offset', # (UTC - GPS offset (secs))
#                 'X',
#                 'Y',
#                 'Z',
#                 'X_dot',
#                 'Y_dot',
#                 'Z_dot',
#                 'YYMMDDhhmmss'],
# #                 usecols = ['name','last_name'],
#                     )




# # del DF_orbit['']
# # X     =  xyzline['X'].values[0].ljust(20)     #'  -745933.8926940708'
# # Y     =  xyzline['Y'].values[0].ljust(20)     #'  -4864983.834066438'
# # Z     =  xyzline['Z'].values[0].ljust(20)     #'    4769954.60524261'
# # X_dot =  xyzline['X_dot'].values[0].ljust(20) #'  457.44564954037634'
# # Y_dot =  xyzline['Y_dot'].values[0].ljust(20) #'   5302.381564886811'
# # Z_dot =  xyzline['Z_dot'].values[0].ljust(20) #'    5463.55571622269'


In [17]:
# DF_orbit

In [18]:
# X = float(DF_orbit["X"].values[0])
# Y = float(DF_orbit["Y"].values[0])
# Z = float(DF_orbit["Z"].values[0])
# X_dot = float(DF_orbit["X_dot"].values[0])
# Y_dot = float(DF_orbit["Y_dot"].values[0])
# Z_dot = float(DF_orbit["Z_dot"].values[0])

# R_vec = [X, Y, Z]
# V_vec = [X_dot, Y_dot, Z_dot]

# dict_OE = RV_to_OE(R_vec, V_vec, mu = 3.986004418*1e14)


In [19]:
# dict_OE